In [3]:
import os 

In [4]:
%pwd

'/home/devil/Text-Summerization-/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/home/devil/Text-Summerization-'

In [7]:
from dataclasses import dataclass
from pathlib import Path

In [44]:
@dataclass(frozen=True)
class DataTransformationConfig: 
    root_dir: Path
    data_path_train: Path
    data_path_test: Path
    data_path_validation: Path
    tokenizer_name: Path

In [45]:
from TextSummerizer.constants import * 
from TextSummerizer.utils.common import read_yaml ,create_directories


In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def data_transformation_config(self) -> DataTransformationConfig: 
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir, 
            data_path_train = config.data_path_train,
            data_path_test = config.data_path_test,
            data_path_validation =  config.data_path_validation,
            tokenizer_name=config.tokenizer_name)

        return data_transformation_config

In [47]:
import os 
from TextSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [52]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['Text'],max_length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['Summary'],max_length=128,truncation=True)

        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_dataset('csv', data_files={'train': self.config.data_path_train, 'test': self.config.data_path_test, 'validation': self.config.data_path_validation})
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "summarizer_dataset"))

In [53]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
    raise

[2023-12-19 03:40:03,719: INFO: common: YAML file 'config/config.yaml' loaded successfully]
[2023-12-19 03:40:03,723: INFO: common: YAML file 'params.yaml' loaded successfully]
[2023-12-19 03:40:03,726: INFO: common: Created directory at: artifacts]
[2023-12-19 03:40:03,730: INFO: common: Created directory at: artifacts/data_transformtaion]


Saving the dataset (1/1 shards): 100%|██████████| 225/225 [00:00<00:00, 23210.00 examples/s]
